In [111]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from icalendar import Calendar, Event, vCalAddress, vText
from datetime import datetime, date, timedelta
import os
import re
import configparser

In [112]:
# Use this to create a config file with you login and password for ut1-capitol
"""
config = configparser.ConfigParser()

config['default'] = {
    'username': '',
    'password': ''
}

with open('config.ini', 'w') as configfile:    # save
    config.write(configfile)
"""

"\nconfig = configparser.ConfigParser()\n\nconfig['default'] = {\n    'username': '',\n    'password': ''\n}\n\nwith open('config.ini', 'w') as configfile:    # save\n    config.write(configfile)\n"

In [113]:
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [114]:
options = Options()
options.add_argument("--headless")
options.binary_location = 'chrome/chrome.exe'
options.add_argument("--window-size=1920,1080")

DRIVER_PATH = 'chromedriver.exe'
driver = webdriver.Chrome(executable_path=DRIVER_PATH, options=options)
driver.implicitly_wait(10)
driver.get('https://cas.ut-capitole.fr/cas/login?service=https%3A%2F%2Fade-production.ut-capitole.fr%2Fdirect%2Fmyplanning.jsp')

In [115]:
user = driver.find_element_by_id("username")
user.send_keys(config.get('default', 'username'))
pas = driver.find_element_by_id("password")
pas.send_keys(config.get('default', 'password'))
login = driver.find_element_by_id("connexion").click()

In [116]:
hours = {}
legend = driver.find_elements_by_class_name("slot")
for l in legend:
    heure = l.text[:-1] if len(l.text) > 5 else l.text
    top = l.get_attribute("style").split("; ")[-1]
    top_nb = int(re.findall(r'\d+', top)[0])+8
    hours[top_nb] = heure

In [117]:
def get_n_weeks(n=4):
    month_french = {
        'January': 'janvier',
        'February': 'février',
        'March': 'mars',
        'April': 'avril',
        'May': 'mai',
        'June': 'juin',
        'July': 'juillet',
        'August': 'août',
        'September': 'septembre',
        'October': 'octobre',
        'November': 'novembre',
        'December': 'décembre'
    }
    today = date.today()
    week = today - timedelta(days=today.weekday())
    n_weeks = []

    for _ in range(n):
        next_day = week.strftime('%d')
        next_month = month_french[week.strftime('%B')]
        next_year = week.strftime('%y')
        n_weeks.append((f'({week.isocalendar().week}){next_day} {next_month} {next_year}', week))
        week+=timedelta(days=7)

    return n_weeks

In [118]:
from time import sleep

cal = Calendar()
n_weeks = get_n_weeks()

duration = {
    32: [1, 0],
    56: [1, 0],
    57: [1, 0],
    84: [1, 30],
    63: [2, 0],
    66: [2, 0],
    112: [2, 0],
    94: [3, 0],
    168: [3, 0],
    167: [3, 0],
    499: [9, 0]
}


for i in range(4):
    current = n_weeks[i][1]
    current_top = 0
    button_week = driver.find_element_by_xpath(f"//button[text()='{n_weeks[i][0]}']")
    button_week.click()
    sleep(10)
    planning = driver.find_element_by_id("Planning")
    planning_div = planning.find_elements_by_xpath("./*")
    planning_data = planning.find_elements_by_xpath("//div[@aria-owns='Planning']")



    for div, data in zip(planning_div, planning_data):
        height_tag = div.find_element_by_tag_name("table").get_attribute("style").split("; ")[-1]
        duration_tag = int(re.findall(r'\d+', height_tag)[0])
        real_duration = duration[duration_tag]

        top = div.get_attribute("style").split("; ")[-1]
        top_nb = int(re.findall(r'\d+', top)[0])
        list_data = [x.strip() for x in list(filter(lambda x: x.replace(" ", "") != "", data.get_attribute("aria-label").split('null')))]

        if current_top >= top_nb:
            current+=timedelta(days=1)
        current_top = top_nb

        year = current.year
        month = current.month
        day = current.day

        hour_class = hours[top_nb].split("h")
        event = Event()
        event.add('summary', list_data[0])
        event.add('dtstart', datetime(year, month, day, int(hour_class[0]), int(hour_class[1])))

        duration_mins = int(hour_class[1])+real_duration[1]
        duration_hours = int(hour_class[0])+real_duration[0]

        if duration_mins == 60:
            duration_hours+=1
            duration_mins=0

        event.add('dtend', datetime(year, month, day, duration_hours, duration_mins))

        event['location'] = vText(list_data[1]if len(list_data) in [4, 5] else 'Inconnu')
        cal.add_component(event)

In [119]:
driver.quit()

In [120]:
f = open(os.path.join("./", 'example.ics'), 'wb')
f.write(cal.to_ical())
f.close()